In [5]:
import pandas as pd
import io
from google.colab import files
import numpy as np

# 1. File Upload
uploaded = files.upload()
filename = list(uploaded.keys())[0]
try:
    df = pd.read_excel(io.BytesIO(uploaded[filename]))
except Exception as e:
    print(f"Error loading Excel file: {e}")
    exit()

# Convert date columns to datetime, handling errors
date_cols = ['contract_date', 'farmer_payment_date', 'plantation_date']  # Add other date columns if needed
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')

def check_data_quality(df):
    report = {}

    # TC-01: Mandatory Fields
    mandatory_cols = ['uid', 'farmer_name', 'District', 'Block']
    report['TC-01_missing_mandatory'] = df[df[mandatory_cols].isnull().any(axis=1)].replace(np.nan, '', regex=True)

    # TC-02: Water/Electricity Availability
    valid_availability = ['Yes', 'No']
    report['TC-02_invalid_availability'] = df[~df['water_available'].isin(valid_availability) | ~df['electricity_available'].isin(valid_availability)].replace(np.nan, '', regex=True)

    # TC-03: Contract-Payment-Plantation Sequence
    report['TC-03_date_sequence'] = df[(df['contract_date'] > df['farmer_payment_date']) | (df['farmer_payment_date'] > df['plantation_date'])].replace(np.nan, '', regex=True)

    # TC-04: Tree Count Validity
    report['TC-04_invalid_tree_count'] = df[(df['trees_planted'] < 350) | (df['trees_planted'] > 450)].replace(np.nan, '', regex=True)

    # TC-05: Document Upload Status
    doc_cols = ['kml_uploaded', 'contract_uploaded', 'land_record_uploaded', 'baseline_survey']
    # Check if all doc_cols are present in df.columns
    doc_cols = [col for col in doc_cols if col in df.columns]
    # If any doc_cols are not in df.columns, a warning will be printed.
    missing_cols = [col for col in doc_cols if col not in df.columns]
    if missing_cols:
        print(f"Warning: The following columns are not found in the DataFrame: {missing_cols}")

    report['TC-05_invalid_doc_status'] = df[df[doc_cols].isin(['Yes', 'No']) == False].replace(np.nan, '', regex=True) # Check if any value is not Yes/No

    # TC-06: Payment Amount Validation
    report['TC-06_invalid_payment'] = df[(df['farmer_payment_collected'] == 'Yes') & (df['amount'] <= 0)].replace(np.nan, '', regex=True)

    # TC-07: Plantation Type Consistency
    valid_plantation_types = ['Fruit Tree', 'Bamboo']
    report['TC-07_invalid_plantation_type'] = df[~df['plantation_type_dense_fruit'].isin(valid_plantation_types)].replace(np.nan, '', regex=True)

    # TC-08: Soil Sample & Training Compliances
    report['TC-08_missing_compliance'] = df[df['plantation_date'].notna() & ((df['soil_sample_collected?'] != 'Yes') | (df['cc_training_uploaded?'] != 'Yes'))].replace(np.nan, '', regex=True)

    # TC-09: Species Distribution Totals
    species_cols = ['mango_native', 'bamboo']  # Add other species columns
    report['TC-09_species_mismatch'] = df[df[species_cols].sum(axis=1) != df['trees_planted']].replace(np.nan, '', regex=True)

    # TC-10: Invalid Dates (already handled during conversion)
    report['TC-10_invalid_dates'] = df[df[date_cols].isna().any(axis=1)].replace(np.nan, '', regex=True) # Check for NaT values after conversion

    return report

quality_report = check_data_quality(df)

# Print the report
for test_case, errors in quality_report.items():
    if not errors.empty:
        print(f"\n--- {test_case} ---")
        print(errors)
    else:
      print(f"\n--- {test_case} ---")
      print("No errors found.")

Saving q1_data.xlsx to q1_data (3).xlsx

--- TC-01_missing_mandatory ---
No errors found.

--- TC-02_invalid_availability ---
    sr_no    uid  program_year farmer_name plantation_type_dense_fruit  \
22     23  id_23          2023    farmer_x                      Bamboo   
23     24  id_24          2023    farmer_y                      Bamboo   

    total_land_area_acre  area_f4f_acre District Block water_available  ...  \
22                     7              4        A     q             Yes  ...   
23                     6              5        B     r             Yes  ...   

   bhendi shirish ain pimpal vad tamhan waval palas babhul bakul  
22      3       0   0      0   0      0     0     0      0     0  
23      5       0   0      0   0      0     0     0      0     0  

[2 rows x 64 columns]

--- TC-03_date_sequence ---
    sr_no    uid  program_year farmer_name plantation_type_dense_fruit  \
1       2   id_2          2023    farmer_c                  Fruit Tree   
2       3   

<ipython-input-5-0e38c19e448b>:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


In [9]:
!pip install --force-reinstall xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 5.4 MB/s eta 0:00:00


# Downloading The Modified XlXS File


In [12]:
import pandas as pd
# Download modified DataFrame as xlsx
from google.colab import files
import io
import os # Import the os module

# Create an output stream
output_stream = io.BytesIO()
# Write the DataFrame to the output stream
with pd.ExcelWriter(output_stream, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)  # index=False to avoid saving the index

# Save the file to disk
with open('modified_data.xlsx', 'wb') as f:
    f.write(output_stream.getbuffer())

# Download the file
files.download('modified_data.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>